# Adversarial (Variational) AutoEncoder

In [6]:
import torch
import torch.nn
import torch.nn.functional as nn
import torch.autograd as autograd
import torch.optim as optim
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.gridspec as gridspec
import os
from torch.autograd import Variable
from tensorflow.examples.tutorials.mnist import input_data

## Hyperparameters

In [7]:
mb_size = 32
z_dim = 5
h_dim = 128
cnt = 0
lr = 1e-3

## Dataset

In [3]:
mnist = input_data.read_data_sets('../dataset/raw', one_hot=True)
X_dim = mnist.train.images.shape[1]
y_dim = mnist.train.labels.shape[1]

Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.
Instructions for updating:
Please write your own downloading logic.
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting ../dataset/raw/train-images-idx3-ubyte.gz
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting ../dataset/raw/train-labels-idx1-ubyte.gz
Instructions for updating:
Please use tf.one_hot on tensors.
Extracting ../dataset/raw/t10k-images-idx3-ubyte.gz
Extracting ../dataset/raw/t10k-labels-idx1-ubyte.gz
Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.


## Architecture

In [4]:
# Encoder
Q = torch.nn.Sequential(
    torch.nn.Linear(X_dim, h_dim),
    torch.nn.ReLU(),
    torch.nn.Linear(h_dim, z_dim)
).cuda()

# Decoder
P = torch.nn.Sequential(
    torch.nn.Linear(z_dim, h_dim),
    torch.nn.ReLU(),
    torch.nn.Linear(h_dim, X_dim),
    torch.nn.Sigmoid()
).cuda()

# Discriminator
D = torch.nn.Sequential(
    torch.nn.Linear(z_dim, h_dim),
    torch.nn.ReLU(),
    torch.nn.Linear(h_dim, 1),
    torch.nn.Sigmoid()
).cuda()

def reset_grad():
    Q.zero_grad()
    P.zero_grad()
    D.zero_grad()

# Training

In [8]:
def sample_X(size, include_y=False):
    X, y = mnist.train.next_batch(size)
    X = Variable(torch.from_numpy(X).cuda())

    if include_y:
        y = np.argmax(y, axis=1).astype(np.int)
        y = Variable(torch.from_numpy(y).cuda())
        return X, y
    return X

Q_solver = optim.Adam(Q.parameters(), lr=lr)
P_solver = optim.Adam(P.parameters(), lr=lr)
D_solver = optim.Adam(D.parameters(), lr=lr)

for it in range(1000000):
    X = sample_X(mb_size)

    """ Reconstruction phase """
    z_sample = Q(X)
    X_sample = P(z_sample)

    recon_loss = nn.binary_cross_entropy(X_sample, X)

    recon_loss.backward()
    P_solver.step()
    Q_solver.step()
    reset_grad()

    """ Regularization phase """
    # Discriminator
    z_real = Variable(torch.randn(mb_size, z_dim).cuda())
    z_fake = Q(X)

    D_real = D(z_real)
    D_fake = D(z_fake)

    D_loss = -torch.mean(torch.log(D_real) + torch.log(1 - D_fake))

    D_loss.backward()
    D_solver.step()
    reset_grad()

    # Generator
    z_fake = Q(X)
    D_fake = D(z_fake)

    G_loss = -torch.mean(torch.log(D_fake))

    G_loss.backward()
    Q_solver.step()
    reset_grad()

    # Print and plot every now and then
    if it % 1000 == 0:
        print('Iter-{}; D_loss: {:.4}; G_loss: {:.4}; recon_loss: {:.4}'
              .format(it, D_loss.item(), G_loss.item(), recon_loss.item()))

        samples = P(z_real).data.cpu().numpy()[:16]

        fig = plt.figure(figsize=(4, 4))
        gs = gridspec.GridSpec(4, 4)
        gs.update(wspace=0.05, hspace=0.05)

        for i, sample in enumerate(samples):
            ax = plt.subplot(gs[i])
            plt.axis('off')
            ax.set_xticklabels([])
            ax.set_yticklabels([])
            ax.set_aspect('equal')
            plt.imshow(sample.reshape(28, 28), cmap='Greys_r')

        if not os.path.exists('out/'):
            os.makedirs('out/')

        plt.savefig('out/{}.png'
                    .format(str(cnt).zfill(3)), bbox_inches='tight')
        cnt += 1
        plt.close(fig)

Iter-0; D_loss: 1.39; G_loss: 0.6931; recon_loss: 0.124
Iter-1000; D_loss: 1.379; G_loss: 0.7002; recon_loss: 0.1611
Iter-2000; D_loss: 1.4; G_loss: 0.6882; recon_loss: 0.1531
Iter-3000; D_loss: 1.393; G_loss: 0.6869; recon_loss: 0.1372
Iter-4000; D_loss: 1.383; G_loss: 0.698; recon_loss: 0.1686
Iter-5000; D_loss: 1.389; G_loss: 0.6885; recon_loss: 0.1441
Iter-6000; D_loss: 1.386; G_loss: 0.7026; recon_loss: 0.1494
Iter-7000; D_loss: 1.384; G_loss: 0.6932; recon_loss: 0.1337
Iter-8000; D_loss: 1.396; G_loss: 0.6913; recon_loss: 0.1614
Iter-9000; D_loss: 1.38; G_loss: 0.6926; recon_loss: 0.1467
Iter-10000; D_loss: 1.384; G_loss: 0.6927; recon_loss: 0.1739
Iter-11000; D_loss: 1.389; G_loss: 0.6864; recon_loss: 0.1607
Iter-12000; D_loss: 1.4; G_loss: 0.6833; recon_loss: 0.1389
Iter-13000; D_loss: 1.39; G_loss: 0.6983; recon_loss: 0.1573
Iter-14000; D_loss: 1.389; G_loss: 0.6967; recon_loss: 0.1578
Iter-15000; D_loss: 1.379; G_loss: 0.6942; recon_loss: 0.1618
Iter-16000; D_loss: 1.384; G_l